# Neural Network

Le **Artificial Neural Network**, (_NN_) sono modelli matematici largamente utilizzati nel campo dell'**Intelligenza Artificiale** (_AI_) che permettono, a sistemi automatici, di compiere task complessi e articolati che dei semplici algoritmi (es. algoritmi sequenziali) non sarebbero in grado di portare a termine in modo rapido ed efficace.

Le basi di questo metodo risalgono alla metà del XX secolo quando per la prima volta furono proposti algoritmi per l'apprendimento automatico. L'obiettivo era quello di creare _strutture_ in grado di modellare un determinato fenomeno e riproporne il comportamento in determinate condizioni.

Il componente base di questa nuova struttura è il _**neurone**_. Con questo termine identifichiamo un nodo in grado di simulare il comportamento di un neurone biologico e di interconnettersi con altri neuroni al fine di creare una rete. Ogni nodo elabora i segnali ricevuti e trasmette il risultato a nodi successivi.

Un tipico esempio di struttura base di queste reti è il **percettrone**:
![Percettrone](./img/percetrone.png)

Ogni singolo ingresso di questi nodi riceve informazioni che vengono elaborate. L'elaborazione, che in base agli ingressi può diventare complessa, si può pensare come singoli ingressi che vengono moltiplicati per un opportuno valore detto peso. Il risultato ottenuto delle moltiplicazioni viene sommato e se la somma supera una certa soglia il neurone attiva la sua uscita. Il peso serve a quantificare l'importanza di una interconnessione, infatti un ingresso molto importante avrà un peso elevato, mentre un ingresso poco utile all'elaborazione avrà un peso inferiore.

Ponendo in cascata e combinando tra loro più più neuroni generiamo quella che definiamo _NN_.
![ANN](./img/ann.png)

L'utilizzo delle NN è tornato in uso dopo la reinvenzione dell’algoritmo di apprendimento chiamato back-propagation. Questo algoritmo infatti permette di modificare i pesi delle interconnessioni in modo tale che si minimizzi una certa funzione errore E.

Con l'avvento di nuove metodologie come il **machine learning** e l'aumento della performance delle NN il campo dell'intelligenza artificiale è diventato tra i più importanti ambiti di ricerca nella computer scienze. Grazie a questo, i risultati e i campi applicativi acquisiscono, di giorno in giorno, maggiore interesse. Siamo così passati ad analizzare, attraverso le _NN_, problemi sempre più complessi: è l'avvento del **deep learning**.

Nelle reti neurali classiche moderne è possibile riscontrare la presenza di qualche strato nascosto. Questi strati, denominati _hidden layers_, possono essere interpretati come il _cuore_ della rete stessa poiché sono quelli che si occupano di interpretare le features sottomesse alla rete.

![NN](./img/simple_neural_network_header.jpg)

Con il deep learning, invece, siamo rapportati a problemi più complessi: _dal riconoscimento ed interpretazione del linguaggio naturale fino alla visione artificiale_. Nei modelli deep ogni singolo stato nascosto potrebbe essere paragonato ad una piccola rete neurale classica: ponendone in cascata una all'altra possiamo ottenere modelli complessi per la gestione di task anche molto avanzati come ad esempio gli algoritmi di visione per l'_automotive_

![deep](./img/deep.png)

#### Codice utlizzato per la Neural Network del caso di studio

In [ ]:
# standard libraries
import os
import shutil
import numpy as np
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout
# keras libraries
from keras.models import Sequential
from keras.models import load_model
from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

# project libraries
from src.nn.keras_nn import load_data_nn

# convolutional network parmas
path_dataset = './dataset/dataset_total.txt'  # datasetpath
path_best = './best_model/'  # kfolds model path
path_thebest = './thebetter_model/'  # bset models path

# neural network params
batch_size = 32  # training cases batch
num_epochs = 500  # max number of epochs
num_classes = 3  # number of class in dataset
seed = 42  # base random seed
n_splits = 10  # number of kfold
n_input_layer = 31 # number of inputs layer

In [10]:
def load_data_nn():
    """
    generate dataset based on data in dataset folder
    :return: train and test dataset based on stratification strategy
    """
    dataset = np.loadtxt(path_dataset, delimiter='\t')

    y = np.array(np.ceil(dataset[:, -1])).astype(np.str)
    X = np.array(dataset[:, :-1]).astype(np.float32)

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=seed, stratify=y)

    scaler = preprocessing.StandardScaler().fit(x_train)

    X_train = scaler.transform(x_train)
    X_test = scaler.transform(x_test)

    y_train = np.subtract(y_train.reshape((len(y_train), 1)).astype(np.float32), np.asarray(2.0))
    y_test = np.subtract(y_test.reshape((len(y_test), 1)).astype(np.float32), np.asarray(2.0))

    Y_train = np_utils.to_categorical(y_train, num_classes)
    Y_test = np_utils.to_categorical(y_test, num_classes)
    return X_train, X_test, Y_train, Y_test

In [12]:
def baseline_model():
    """
    Definition of neural network base model
    :return: model
    """
    base_model = Sequential()
    base_model.add(Dense(n_input_layer, activation='elu',  input_shape=(X_train.shape[1],)))
    #hidden
    base_model.add(Dense(31, activation='elu'))
    base_model.add(Dropout(0.1))
    base_model.add(Dense(31, activation='elu'))
    base_model.add(Dropout(0.1))
    base_model.add(Dense(31, activation='elu'))
    base_model.add(Dropout(0.1))
    base_model.add(Dense(31, activation='elu'))
    base_model.add(Dropout(0.3))
    base_model.add(Dense(3, activation='softmax'))
    base_model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    return base_model

In [13]:
def evaluete_nn(X_test, Y_test, best_model):
    """
    Evaluate best model after kfold training
    :param X_test: example images to test best model after kfold
    :param Y_test: labels matching truth to example
    :param best_model: index which identify best model after kfold
    :return: evaluation of best model trough dataset test and save it with loss and accuracy metrics
    """

    # load best model and evaluate it with accuracy and loss
    print('Load best model and test it ')
    model = load_model(path_best+'checkpoint-%d.h5' %(best_model))
    score = model.evaluate(X_test, Y_test, verbose=0)  # evaluate model
    print('Metrics => ', model.metrics_names, score)
    y_predict = np.asarray(model.predict(X_test, verbose=0))
    Y_predict = np.argmax(y_predict, axis=1)
    y_test = np.argmax(Y_test, axis=1)
    confmatrix = confusion_matrix(y_test, Y_predict)
    print("\nConfusion Matrix :")
    print(confmatrix)
    class_names = ["0", "1", '2']
    print("\nMetrics => ", model.metrics_names, score)
    print('\nClassification Report : ')
    print(classification_report(y_test, Y_predict, target_names=class_names))
    # save model tested with loss and accuracy
    model.save(path_thebest+'model-'+'{:.4f}'.format(score[0])+'-'+'{:.4f}'.format(score[1])+'.h5')

In [14]:
print('STARTING FITTING NEURAL NETWORK')
if os.path.exists(path_best):
    shutil.rmtree(path_best)
os.mkdir(path_best)
if not os.path.exists(path_thebest):
    os.mkdir(path_thebest)

X_train, X_test, Y_train, Y_test = load_data_nn()

print('loading data .......')

STARTING FITTING NEURAL NETWORK
loading data .......


In [15]:
# generation kfolds to cross validation process
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
cvscores = []
i = 0

# start cross validation
for train, test in kfold.split(X_train, Y_train):
    model = baseline_model()
    model.fit(X_train[train], Y_train[train], epochs=num_epochs, batch_size=batch_size, verbose=0,
              callbacks=[TensorBoard(log_dir='./nn/tensorboard/'),
                         ModelCheckpoint(path_best+'checkpoint-%d.h5' %(i), monitor='acc', verbose=0,
                                         save_best_only=True, mode='max'),
                         EarlyStopping(monitor='loss', min_delta=0.001, patience=10, verbose=2, mode='min')])
    scores = model.evaluate(X_train[test], Y_train[test], verbose=2)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    i += 1
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 00052: early stopping
acc: 96.13%
Epoch 00041: early stopping
acc: 97.50%
Epoch 00056: early stopping
acc: 95.76%
Epoch 00037: early stopping
acc: 96.88%
Epoch 00037: early stopping
acc: 96.50%
Epoch 00054: early stopping
acc: 96.25%
Epoch 00031: early stopping
acc: 94.51%
Epoch 00046: early stopping
acc: 96.88%
Epoch 00041: early stopping
acc: 96.38%
Epoch 00056: early stopping
acc: 97.25%
96.40% (+/- 0.81%)
Load best model and test it 
Metrics =>  ['loss', 'acc'] [0.095308599793494528, 0.96855895196506547]

Confusion Matrix :
[[1077    0    1]
 [  18    7    1]
 [  34    0    7]]

Metrics =>  ['loss', 'acc'] [0.095308599793494528, 0.96855895196506547]

Classification Report : 
             precision    recall  f1-score   support

          0       0.95      1.00      0.98      1078
          1       1.00      0.27      0.42        26
          2       0.78      0.17      0.28        41

avg / total       0.95      0.95      0.94      1145



In [16]:
# evaluate best model based on higher accuracy
vect_max = np.argmax(cvscores)
evaluete_nn(X_test, Y_test, vect_max)

Load best model and test it 
Metrics =>  ['loss', 'acc'] [0.095308599793494528, 0.96855895196506547]

Confusion Matrix :
[[1077    0    1]
 [  18    7    1]
 [  34    0    7]]

Metrics =>  ['loss', 'acc'] [0.095308599793494528, 0.96855895196506547]

Classification Report : 
             precision    recall  f1-score   support

          0       0.95      1.00      0.98      1078
          1       1.00      0.27      0.42        26
          2       0.78      0.17      0.28        41

avg / total       0.95      0.95      0.94      1145

